## Imports

In [1]:
import os
from dotenv import load_dotenv

import polars as pl
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns



## Config

In [2]:
# Fix random state for reproducibility
RANDOM_STATE = 42

# Load environment variables
load_dotenv()

# Get project root (one level up from notebooks/)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..')) if 'notebooks' in os.getcwd() else os.getcwd()

# Get preprocessed directory from env or use default
preprocessed_dir = os.getenv('PREPROCESSED_DATA_DIR', 'data/preprocessed')

# If path is relative, make it relative to project root
if not os.path.isabs(preprocessed_dir):
    preprocessed_dir = os.path.join(project_root, preprocessed_dir)

# Check if preprocessed data exists
print(f'📁 Project root: {project_root}')
print(f'📁 Preprocessed dir: {preprocessed_dir}')

if not os.path.exists(preprocessed_dir):
    print(f'\n⚠️  Warning: Preprocessed directory not found!')
    print('   Run from project root: python3 -m src.main')
else:
    print(f'\n✅ Preprocessed directory found')


📁 Project root: /home/mle-user/mle_projects/mle-project-sprint-4
📁 Preprocessed dir: /home/mle-user/mle_projects/mle-project-sprint-4/data/preprocessed

✅ Preprocessed directory found


## Preprocessed Data Loading

In [3]:
# ---------- Load datasets ---------- #

# Check if preprocessed files exist
items_path = os.path.join(preprocessed_dir, 'items.parquet')
events_path = os.path.join(preprocessed_dir, 'events.parquet')
tracks_catalog_path = os.path.join(preprocessed_dir, 'tracks_catalog_clean.parquet')

if not os.path.exists(items_path) or not os.path.exists(events_path):
    print('❌ Preprocessed data not found!')
    print(f'   Looking for files in: {preprocessed_dir}')
    print('\n📝 To generate preprocessed data, run from terminal:')
    print('   cd /home/mle-user/mle_projects/mle-project-sprint-4')
    print('   python3 -m src.main')
    print('\n   This will:')
    print('   1. Download raw data (if needed)')
    print('   2. Preprocess and clean the data')
    print('   3. Generate items.parquet and events.parquet')
    raise FileNotFoundError(f'Preprocessed data not found in {preprocessed_dir}')

# Load preprocessed data
print('✅ Loading preprocessed data...')
items = pl.read_parquet(items_path)
events = pl.read_parquet(events_path)
tracks_catalog_clean = pl.read_parquet(tracks_catalog_path)

print(f'\n✅ Data loaded successfully!')
print(f'   Items: {items.shape}')
print(f'   Events: {events.shape}')
print(f'   Tracks catalog: {tracks_catalog_clean.shape}')

✅ Loading preprocessed data...

✅ Data loaded successfully!
   Items: (5575922, 9)
   Events: (205866117, 4)
   Tracks catalog: (804866, 6)


## EDA

In [4]:
# ---------- Check data summary ---------- #

def data_summary(df: pl.DataFrame, name: str):
    '''
    Display a quick overview of a Polars DataFrame.
    '''

    print(f'\n===== {name.upper()} =====')  
  
    # Sample rows
    print('\nSample rows:')
    display(df.head())

    # Shape
    rows, cols = df.shape
    print(f'\nShape: {rows:,} rows x {cols} columns')
    
    # Data info
    print('\nSummary for numeric columns:')
    display(df.describe())
   
    # Column info
    print('\nColumn names and types:')
    for col in df.columns:
        print(f'  {col}: {df[col].dtype}')
    
    # Missing values
    print('\nMissing values:')
    display(df.null_count())

In [5]:
# ---------- Items data summary ---------- #

data_summary(items, 'items')


===== ITEMS =====

Sample rows:


track_id,artist_id,album_id,genre_id,track_clean,track_group_id,artist_clean,album_clean,genre_clean
i64,i64,i64,i64,str,i64,str,str,str
14470902,65471,1579490,68,"""Fornax""",14470902,"""Smooth""","""The Sky S Gonna Break""","""Electronics"""
14145383,1654436,17031710,11,"""Royals""",14145383,"""Lorde""","""Girl Power""","""Pop"""
14571551,35544,2120247,335,"""Just For Today I Will Respect""",14571551,"""Llewellyn""","""Tantric Love Music For Tantric…","""Relax"""
2836669,3989,2726384,28,"""Night And Day""",2836669,"""Frank Sinatra""","""My Special Angel""","""Jazz"""
14377823,375233,12628948,69,"""Chopin Nocturne No 20 In C Sha…",14377823,"""Maria Joao Pires""","""Chopin Urban Autumn""","""Classicalmusic"""



Shape: 5,575,922 rows x 9 columns

Summary for numeric columns:


statistic,track_id,artist_id,album_id,genre_id,track_clean,track_group_id,artist_clean,album_clean,genre_clean
str,f64,f64,f64,f64,str,f64,str,str,str
"""count""",5.575922e6,5.575922e6,5.575922e6,5.575922e6,"""5575922""",5.575922e6,"""5575922""","""5575922""","""5575922"""
"""null_count""",0.0,0.0,0.0,0.0,"""0""",0.0,"""0""","""0""","""0"""
"""mean""",2.5683e7,1.7932e6,6.0408e6,79.057782,null,2.5669e7,null,null,null
"""std""",2.2892e7,2.2239e6,5.4589e6,119.323574,null,2.2889e7,null,null,null
"""min""",26.0,4.0,3.0,1.0,"""""",26.0,"""""","""""","""African"""
"""25%""",4.515241e6,49502.0,2.529212e6,19.0,null,4.505449e6,null,null,null
"""50%""",2.3531749e7,664478.0,3.362319e6,49.0,null,2.3531749e7,null,null,null
"""75%""",3.5855375e7,3.343208e6,9.746128e6,74.0,null,3.58377e7,null,null,null
"""max""",1.01495927e8,1.6102782e7,2.1458968e7,1542.0,"""Ꞌcosmicꞌ M4A""",1.01495927e8,"""龍胆紫 Purplesoul""","""Ꞌcosmicꞌ M4A""","""Work"""



Column names and types:
  track_id: Int64
  artist_id: Int64
  album_id: Int64
  genre_id: Int64
  track_clean: String
  track_group_id: Int64
  artist_clean: String
  album_clean: String
  genre_clean: String

Missing values:


track_id,artist_id,album_id,genre_id,track_clean,track_group_id,artist_clean,album_clean,genre_clean
u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0


In [6]:
# ---------- Events data summary ---------- #

data_summary(events, 'events')


===== EVENTS =====

Sample rows:


user_id,track_id,listen_count,last_listen
i32,i64,u32,date
1207,67944660,1,2022-11-23
1209,51824040,1,2022-12-08
1213,61557165,1,2022-11-24
1216,43202169,1,2022-09-01
1217,85605287,1,2022-11-09



Shape: 205,866,117 rows x 4 columns

Summary for numeric columns:


statistic,user_id,track_id,listen_count,last_listen
str,f64,f64,f64,str
"""count""",2.05866117e8,2.05866117e8,2.05866117e8,"""205866117"""
"""null_count""",0.0,0.0,0.0,"""0"""
"""mean""",687069.08984,3.2761e7,1.025474,"""2022-08-29 02:39:16.619557"""
"""std""",396888.652222,2.7494e7,0.174055,null
"""min""",0.0,26.0,1.0,"""2022-01-01"""
"""25%""",342929.0,3.598724e6,1.0,"""2022-07-01"""
"""50%""",687341.0,3.1156067e7,1.0,"""2022-09-14"""
"""75%""",1.030392e6,5.3705229e7,1.0,"""2022-11-09"""
"""max""",1.374582e6,1.01495927e8,24.0,"""2022-12-31"""



Column names and types:
  user_id: Int32
  track_id: Int64
  listen_count: UInt32
  last_listen: Date

Missing values:


user_id,track_id,listen_count,last_listen
u32,u32,u32,u32
0,0,0,0


In [7]:
# tracks_catalog_clean already loaded in Cell 5
display(tracks_catalog_clean.head())

track_id,track_clean,track_group_id,artist_id,album_id,genre_id
i64,str,i64,i64,i64,i64
5731590,"""The Kids Are Alright""",5731590,4481,2506226,14
15328375,"""Ciao Amore Ciao""",15328375,1265,6299493,11
650750,"""Spanish Guitar""",650750,34217,73539,11
15331201,"""Музыка Венчальная""",15331201,219052,1674979,20
9443150,"""Tchaikovsky The Nutcracker Op …",9443150,1457816,18428544,117


In [9]:
#---------- Top tracks by popularity ---------- #

top_tracks_by_listen_number = (
    events
        .group_by('track_id')
        .agg(pl.sum('listen_count').alias('total_listen_count'))
        .join(tracks_catalog_clean.select(['track_id', 'track_clean']), on='track_id', how='left')
        .sort('total_listen_count', descending=True)
        .head(10)
)

display(top_tracks_by_listen_number)

track_id,total_listen_count,track_clean
i64,u32,str
40330534,124756,"""In The End"""
78426489,122267,"""Rampampam"""
53404,121463,"""Smells Like Teen Spirit"""
33311009,107301,"""Believer"""
251849,105740,"""The Show Must Go On"""
178477,102711,"""Numb"""
24692821,100167,"""Way Down We Go"""
795836,99350,"""Shape Of My Heart"""
35505245,98962,"""I Got Love"""


In [10]:
# Memory-efficient approach: aggregate first (small result), then join for names
print("=== Top Track Versions ===")

# Step 1: Get top 10 track IDs by listen count (small DataFrame)
top_track_ids = (
    events
    .lazy()
    .group_by('track_id')
    .agg(pl.sum('listen_count').alias('total_listens'))
    .sort('total_listens', descending=False)
    .tail(10)  # Use tail for descending (more efficient)
    .collect()
)

# Step 2: Join only these 10 tracks with catalog (small join)
top_track_versions = (
    top_track_ids
    .join(tracks_catalog_clean.select(['track_id', 'track_clean', 'track_group_id']), 
          on='track_id', how='left')
    .sort('total_listens', descending=True)
)
display(top_track_versions)

# 2. Most popular songs (versions aggregated) - memory efficient
print("\n=== Top Songs (All Versions Combined) ===")

# Step 1: Aggregate by track_id first (reduces size before join)
events_agg = (
    events
    .lazy()
    .group_by('track_id')
    .agg(pl.sum('listen_count').alias('track_listens'))
    .collect()
)

# Step 2: Join with catalog (smaller join)
events_with_groups = (
    events_agg
    .join(tracks_catalog_clean.select(['track_id', 'track_clean', 'track_group_id']), 
          on='track_id', how='left')
)

# Step 3: Group by track_group_id
top_songs = (
    events_with_groups
    .group_by(['track_group_id', 'track_clean'])
    .agg([
        pl.sum('track_listens').alias('total_listens'),
        pl.len().alias('num_versions'),
    ])
    .sort('total_listens', descending=True)
    .head(10)
)
display(top_songs)

# 3. Analysis: How many top tracks are just different versions?
print("\n=== Diversity Analysis ===")

# Get top 100 tracks
top_100_track_ids = (
    events
    .lazy()
    .group_by('track_id')
    .agg(pl.sum('listen_count').alias('total_listens'))
    .sort('total_listens', descending=False)
    .tail(100)
    .collect()
)

top_100_with_groups = (
    top_100_track_ids
    .join(tracks_catalog_clean.select(['track_id', 'track_group_id']), 
          on='track_id', how='left')
)

unique_groups_in_top_100 = top_100_with_groups['track_group_id'].n_unique()
print(f"Top 100 tracks represent {unique_groups_in_top_100} unique songs")
print(f"Duplication rate: {(100 - unique_groups_in_top_100) / 100:.1%}")

=== Top Track Versions ===


track_id,total_listens,track_clean,track_group_id
i64,u32,str,i64
40330534,124756,"""In The End""",40330534
78426489,122267,"""Rampampam""",78426489
53404,121463,"""Smells Like Teen Spirit""",53404
33311009,107301,"""Believer""",33311009
251849,105740,"""The Show Must Go On""",251849
178477,102711,"""Numb""",178477
24692821,100167,"""Way Down We Go""",24692821
795836,99350,"""Shape Of My Heart""",795836
35505245,98962,"""I Got Love""",35505245



=== Top Songs (All Versions Combined) ===


track_group_id,track_clean,total_listens,num_versions
i64,str,u32,u32
40330534,"""In The End""",124756,1
78426489,"""Rampampam""",122267,1
53404,"""Smells Like Teen Spirit""",121463,1
33311009,"""Believer""",107301,1
251849,"""The Show Must Go On""",105740,1
178477,"""Numb""",102711,1
24692821,"""Way Down We Go""",100167,1
795836,"""Shape Of My Heart""",99350,1
35505245,"""I Got Love""",98962,1



=== Diversity Analysis ===
Top 100 tracks represent 100 unique songs
Duplication rate: 0.0%


Наиболее популярные жанры

In [11]:
# Top 5 genres by listening number - memory efficient
# Step 1: Aggregate events by track_id (reduces size)
events_by_track = (
    events
    .lazy()
    .group_by('track_id')
    .agg(pl.sum('listen_count').alias('track_listen_count'))
    .collect()
)

# Step 2: Get unique track-genre mapping (small subset of items)
track_genres = items.select(['track_id', 'genre_clean']).unique(['track_id', 'genre_clean'])

# Step 3: Join and aggregate
genres_by_listen_count = (
    events_by_track
    .join(track_genres, on='track_id', how='left')
    .group_by('genre_clean')
    .agg(pl.sum('track_listen_count').alias('total_listen_count'))
    .sort('total_listen_count', descending=True)
)

top_5_genres = genres_by_listen_count.head(5)
display(top_5_genres)

genre_clean,total_listen_count
str,u32
"""Pop""",65937521
"""Rap""",38378370
"""Allrock""",34127931
"""Ruspop""",28813305
"""Rusrap""",25351340


Треки, которые никто не прослушал

In [12]:
# Tracks that haven't been listened to by anybody - memory efficient
# Step 1: Get set of listened track IDs (small set)
listened_track_ids = set(events['track_id'].unique())

# Step 2: Get unique tracks from items
unique_tracks = items.select(['track_id', 'track_clean', 'artist_clean', 'album_clean', 'genre_clean']).unique('track_id')

# Step 3: Filter out listened tracks (using Python set for efficiency)
unlistened_tracks = unique_tracks.filter(~pl.col('track_id').is_in(list(listened_track_ids)))

print(f'Number of unlistened tracks: {unlistened_tracks.height:,}')
display(unlistened_tracks.head(10))

Number of unlistened tracks: 49


track_id,track_clean,artist_clean,album_clean,genre_clean
i64,str,str,str,str
61606834,"""В Любои Момент Могу Бросить""","""Вбензинеестьоблака""","""Все Что Вы Хотели Но Боялись Е…","""Rap"""
73459486,"""Lord Of The Flies""","""Elton John""","""Jewel Box""","""Pop"""
70050701,"""Symphonic Suite Kiki S Deliver…","""Joe Hisaishi""","""Symphonic Suite Kiki S Deliver…","""Classicalmusic"""
56166858,"""Панк Туристы""","""Скамеика Запасных""","""Новые Грехи Старые Традиции""","""Punk"""
77751634,"""Gypsy Drum Solo""","""Dio""","""Holy Diver""","""Classicmetal"""
57627360,"""Statues""","""Death From Above 1979""","""Outrage Is Now""","""Alternative"""
46786298,"""Heresy Blind""","""Nine Inch Nails""","""Closer To God""","""Metal"""
88466656,"""Last Horizon""","""Brian May""","""Back To The Light""","""Rock"""
97264237,"""Рождество""","""Арт Группа Largo""","""Рождество""","""Spiritual"""
